## 케라스에서도 scikit-learn의 GridsearchCV, randomizedsearchCV 사용하기


### scikit-keras 사용

In [ ]:
# scikeras 설치
# %pip install scikeras[tensorflow]

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import VGG16
from keras.utils import plot_model # 모델 플롯 그리기
from keras.utils import set_random_seed # 랜덤 시드 설정
from keras.models import save_model, load_model # 모델 저장 및 로드
from sklearn.decomposition import PCA, NMF
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import classification_report

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()
data = iris['data']
target = iris['target']
name = iris['feature_names']

In [5]:
# 표준화 작업
st = StandardScaler()
st.fit(data)
data = st.transform(data)

In [6]:
# train, test, val 나눠서 작업
train_data, test_data, train_target, test_target = train_test_split(data, target, stratify=target, random_state=42, test_size=0.2)
train_data2, val_data, train_target2, val_target = train_test_split(train_data, train_target, stratify=train_target, random_state=42, test_size=0.2)

In [10]:
md = Sequential()
md.add(Dense(16,activation='relu', input_shape=(4,)))
md.add(Dropout(0.3))
md.add(Dense(32,activation='relu'))
md.add(Dropout(0.3))
md.add(Dense(64,activation='relu'))
md.add(Dropout(0.3))
md.add(Dense(3, activation='softmax'))
md.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
md.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                80        
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                544       
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 64)                2112      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                

In [12]:
mcp = ModelCheckpoint(".",monitor='val_accuracy', save_best_only=True)
history = md.fit(train_data2, train_target2, epochs=100, batch_size=16, validation_data=(val_data, val_target), callbacks=[mcp])

Epoch 1/100
6/6 [==============================] - 1s 287ms/step - loss: 0.4423 - accuracy: 0.8229 - val_loss: 0.3484 - val_accuracy: 0.7917
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 0.4124 - accuracy: 0.8333 - val_loss: 0.3382 - val_accuracy: 0.7917
Epoch 3/100
6/6 [==============================] - 0s 9ms/step - loss: 0.4093 - accuracy: 0.8333 - val_loss: 0.3305 - val_accuracy: 0.7917
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 0.3831 - accuracy: 0.8750 - val_loss: 0.3250 - val_accuracy: 0.7917
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 0.3470 - accuracy: 0.8750 - val_loss: 0.3176 - val_accuracy: 0.7917
Epoch 6/100
6/6 [==============================] - 0s 8ms/step - loss: 0.3581 - accuracy: 0.8542 - val_loss: 0.3083 - val_accuracy: 0.7917
Epoch 7/100
6/6 [==============================] - 1s 244ms/step - loss: 0.3948 - accuracy: 0.8542 - val_loss: 0.2983 - val_accuracy: 0.8750
Epoch 8/100
6/6 [======

In [16]:
print(md.evaluate(test_data, test_target))
print(mcp.model.evaluate(test_data, test_target))

1/1 [==============================] - 0s 29ms/step - loss: 0.2274 - accuracy: 0.9000
[0.2273644655942917, 0.8999999761581421]
1/1 [==============================] - 0s 23ms/step - loss: 0.2274 - accuracy: 0.9000
[0.2273644655942917, 0.8999999761581421]


### GridSearchCV 이용해서 최적의 해 찾기

In [35]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

clf = KerasClassifier(md, loss='sparse_categorical_crossentropy')

In [28]:
params = {
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, cv=3, scoring='accuracy', refit=True, return_train_score=True, n_jobs=-1, verbose=0)

gs.fit(train_data2, train_target2)
print(gs.best_score_, gs.best_params_)

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_djtpuz2\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmptno8o3xb\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpg3yajkrs\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp6jeehdmc\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_76xqfwx\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpsi57ej6y\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp8s3zco2n\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpsv6vgcve\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpt3ap0ddr\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp0gac5ac9\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_mvolsld\assets
INFO:tensorflow:Assets written to: C:\Users

In [33]:
y_pred = gs.best_estimator_.predict(test_data)

1/1 [==============================] - 0s 25ms/step


In [34]:
print(classification_report(test_target,y_pred) )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.80      0.84        10
           2       0.82      0.90      0.86        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



### RandomSearchCV 써보기

In [36]:
params = {
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
rs = RandomizedSearchCV(clf, params, cv=3, scoring='accuracy', refit=True, return_train_score=True, n_jobs=-1, verbose=0)

rs.fit(train_data2, train_target2)
print(rs.best_score_, rs.best_params_)

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpictnn2m0\assets


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmptct1rrxo\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp6ju424vf\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpls4t1ah3\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmppl6_mrdu\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpzk15f9ds\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp3c4ajkka\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpcbcghlun\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp0vdk2hkx\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp4cyjnt2g\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpv9ang_3e\assets
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpudhk5cwb\assets
INFO:tensorflow:Assets written to: C:\Users

### GridSearch 보완

In [43]:
def get_model(n_hidden, n_neurons, learning_rate, drop_rate, input_shape=(4,)):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    model = Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=input_shape))
    for i in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
        model.add(Dropout(drop_rate))
    model.add(Dense(3, activation='softmax'))
    return model
clf = KerasClassifier(get_model, n_hidden=3, n_neurons=28, learning_rate=0.001, drop_rate=0.3, epochs=50, loss='sparse_categorical_crossentropy')

In [45]:
params = {
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "learning_rate": [0.0001, 0.001, 0.1],
    "n_neurons": [16,24,32,40,48,56,64],
    "n_hidden": [1,2,3,4,5,6,7,8,9,10,11,12],
    "drop_rate":[0.3,0.4,0.5]
}
rs = RandomizedSearchCV(clf, params, cv=3, scoring='accuracy', refit=True, return_train_score=True, n_jobs=-1, verbose=0)

rs.fit(train_data2, train_target2)
print(rs.best_score_, rs.best_params_)

ValueError: Invalid parameter drop_rate for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(drop_rate=0.4)`
Check the list of available parameters with `estimator.get_params().keys()`